In [ ]:
pip install knowledge-storm

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
pip install --upgrade openai

In [ ]:
import os
from knowledge_storm import STORMWikiRunnerArguments, STORMWikiRunner, STORMWikiLMConfigs
from knowledge_storm.lm import OpenAIModel
from knowledge_storm.rm import YouRM

In [ ]:
os.environ["OPENAI_API_KEY"]="your_api_key_here"
os.environ["YDC_API_KEY"]="your_YDC_api_key_here"
lm_configs = STORMWikiLMConfigs()
openai_kwargs = {
    'api_key': os.getenv("OPENAI_API_KEY"),
    'temperature': 1.0,
    'top_p': 0.9,
}

In [ ]:
# STORM is a LM system so different components can be powered by different models to reach a good balance between cost and quality.
# For a good practice, choose a cheaper/faster model for `conv_simulator_lm` which is used to split queries, synthesize answers in the conversation.
# Choose a more powerful model for `article_gen_lm` to generate verifiable text with citations.
gpt_35 = OpenAIModel(model='gpt-4o-mini', max_tokens=500, **openai_kwargs)#this was gpt-3.5-turbo earlier
gpt_4 = OpenAIModel(model='gpt-4o-mini', max_tokens=3000, **openai_kwargs)#this was gpt-4-turbo earlier
lm_configs.set_conv_simulator_lm(gpt_35)
lm_configs.set_question_asker_lm(gpt_35)
lm_configs.set_outline_gen_lm(gpt_4)
lm_configs.set_article_gen_lm(gpt_4)
lm_configs.set_article_polish_lm(gpt_4)
# Check out the STORMWikiRunnerArguments class for more configurations.
# Initialize STORMWikiRunnerArguments with required parameters
engine_args = STORMWikiRunnerArguments(output_dir='output') # Replace 'output' with desired output directory
rm = YouRM(ydc_api_key=os.getenv('YDC_API_KEY'), k=engine_args.search_top_k)
runner = STORMWikiRunner(engine_args, lm_configs, rm)

In [ ]:
topic = input('Topic: ')
runner.run(
    topic=topic,
    do_research=True,
    do_generate_outline=True,
    do_generate_article=True,
    do_polish_article=True,
)
runner.post_run()
runner.summary()

Topic: Short Stories of Leo Tolstoy


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

***** Execution time *****
run_knowledge_curation_module: 54.0739 seconds
run_outline_generation_module: 6.9867 seconds
run_article_generation_module: 17.3569 seconds
run_article_polishing_module: 4.1120 seconds
***** Token usage of language models: *****
run_knowledge_curation_module
    gpt-4o-mini: {'prompt_tokens': 17436, 'completion_tokens': 6546}
run_outline_generation_module
    gpt-4o-mini: {'prompt_tokens': 776, 'completion_tokens': 555}
run_article_generation_module
    gpt-4o-mini: {'prompt_tokens': 7363, 'completion_tokens': 2059}
run_article_polishing_module
    gpt-4o-mini: {'prompt_tokens': 2076, 'completion_tokens': 397}
***** Number of queries of retrieval models: *****
run_knowledge_curation_module: {'YouRM': 36}
run_outline_generation_module: {'YouRM': 0}
run_article_generation_module: {'YouRM': 0}
run_article_polishing_module: {'YouRM': 0}
